In [138]:
import os
import pandas as pd
data_path = "/Users/abhishekkumar/Documents/MS_UB/Fall_18/ESL506_StatisticalDataMiningI/Project/SIGNATE/translated/"

train = pd.read_table(os.path.join(data_path,"'ttrain.tsv'.txt"))
test = pd.read_table(os.path.join(data_path,"'ttest.tsv'.txt"))
######################################################################
#Transforming Training Data
#####################################################################
#One hot encoding for parks
train_df = pd.DataFrame()
train_df["Akan"] = train["epark"] == "Akan Megumi National Park"
train_df["Aso"] = train["epark"] == "Aso Kuju National Park"
train_df["Ise"] = train["epark"] == "Ise Shima National Park"
train_df["Kerama"] = train["epark"] == "Kerama islands national park"
train_df["Kirishima"] = train["epark"] == "Kirishima Jinjiang Bay National Park"
train_df["Nikko"] = train["epark"] == "Nikko National Park"
train_df["Oyama"] = train["epark"] == "Oyama Oki National Park"
train_df["Towada"] = train["epark"] == "Towada Hachimantai National Park"

#
train['datetime'] = pd.to_datetime(train['datetime'])

#Creating new features with datetime
train_df["dayofweek"] = train['datetime'].dt.dayofweek
train_df["dayofweek"] = train_df["dayofweek"].astype('object')
train_df["month"] = train['datetime'].dt.month
train_df["month"] = train_df["month"].astype('object')
train_df["year"] = train['datetime'].dt.year
train_df["year"] = train_df["year"].astype('object')
train_df["day"] = train['datetime'].dt.day
train_df["day"] = train_df["day"].astype('object')



##############################################################################
#Transforming Test Data
#############################################################################
#One hot encoding for parks
test_df = pd.DataFrame()
test_df["Akan"] = test["epark"] == "Akan Megumi National Park"
test_df["Aso"] = test["epark"] == "Aso Kuju National Park"
test_df["Ise"] = test["epark"] == "Ise Shima National Park"
test_df["Kerama"] = test["epark"] == "Kerama islands national park"
test_df["Kirishima"] = test["epark"] == "Kirishima Jinjiang Bay National Park"
test_df["Nikko"] = test["epark"] == "Nikko National Park"
test_df["Oyama"] = test["epark"] == "Oyama Oki National Park"
test_df["Towada"] = test["epark"] == "Towada Hachimantai National Park"

#
test['datetime'] = pd.to_datetime(train['datetime'])

#Creating new features with datetime
test_df["dayofweek"] = test['datetime'].dt.dayofweek
test_df["dayofweek"] = test_df["dayofweek"].astype('object')
test_df["month"] = test['datetime'].dt.month
test_df["month"] = test_df["month"].astype('object')
test_df["year"] = test['datetime'].dt.year
test_df["year"] = test_df["year"].astype('object')
test_df["day"] = test['datetime'].dt.day
test_df["day"] = test_df["day"].astype('object')

In [139]:
train_df.head()

,Akan,Aso,Ise,Kerama,Kirishima,Nikko,Oyama,Towada,dayofweek,month,year,day
0,True,False,False,False,False,False,False,False,3,1,2015,1
1,True,False,False,False,False,False,False,False,4,1,2015,2
2,True,False,False,False,False,False,False,False,5,1,2015,3
3,True,False,False,False,False,False,False,False,6,1,2015,4
4,True,False,False,False,False,False,False,False,0,1,2015,5


In [140]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier
import xgboost as xgb

In [150]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))


y = train['visitors'].values
X = train_df

#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=45)

In [175]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
import datetime
from sklearn.model_selection import GridSearchCV

now = datetime.datetime.now()

# Load the data


y_train_full = y
x_train_full = X

#x_test = X_test

# Convert columns that are not numeric to a numeric value
for c in x_train_full.columns:
    if x_train_full[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_train_full[c].values))
        x_train_full[c] = lbl.transform(list(x_train_full[c].values))
        # x_train_full.drop(c,axis=1,inplace=True)

for c in x_test.columns:
    if x_test[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_test[c].values))
        x_test[c] = lbl.transform(list(x_test[c].values))
        # x_test.drop(c,axis=1,inplace=True)



        
# Various hyper-parameters to tune
xgb1 = XGBRegressor()
gama = np.linspace(0.1,0.5,9)
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [0.01, 0.1], #so called `eta` value
              'max_depth': [ 6],
              'min_child_weight': [3, 5, 7],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.6, 0.7],
              'n_estimators': [1000],
              'reg_lambda':[0.1, 0.3],
              'gamma': gama}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 5,
                        n_jobs = 5,
                        verbose=True)

model = xgb_grid.fit(x_train_full,
         y_train_full, eval_metric='mae')

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:   32.2s
[Parallel(n_jobs=5)]: Done 190 tasks      | elapsed:  2.4min
[Parallel(n_jobs=5)]: Done 440 tasks      | elapsed:  5.5min
[Parallel(n_jobs=5)]: Done 790 tasks      | elapsed: 10.7min
[Parallel(n_jobs=5)]: Done 1080 out of 1080 | elapsed: 15.3min finished


0.3842838891415538
{'colsample_bytree': 0.8, 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 6, 'min_child_weight': 7, 'n_estimators': 800, 'nthread': 4, 'objective': 'reg:linear', 'reg_lambda': 0.1, 'silent': 1, 'subsample': 0.7}


In [176]:
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
score = np.mean(abs(y_pred - y_test))
print(score)

966.0395677138598


In [177]:
import pandas as pd

final_test = test_df
for c in final_test.columns:
    if final_test[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(final_test[c].values))
        final_test[c] = lbl.transform(list(final_test[c].values))
        
y_pred = model.predict(final_test)


In [186]:
set(test["epark"])

{'Akan Megumi National Park',
 'Aso Kuju National Park',
 'Ise Shima National Park',
 'Kerama islands national park',
 'Kirishima Jinjiang Bay National Park',
 'Nikko National Park',
 'Oyama Oki National Park',
 'Towada Hachimantai National Park'}

In [189]:
akan_mean = np.mean(train[train["epark"]=="Akan Megumi National Park"]["visitors"])
aso_mean = np.mean(train[train["epark"]=="Aso Kuju National Park"]["visitors"])
ise_mean = np.mean(train[train["epark"]=="Ise Shima National Park"]["visitors"])
kerama_mean = np.mean(train[train["epark"]=="Kerama islands national park"]["visitors"])
kirishima_mean = np.mean(train[train["epark"]=="Kirishima Jinjiang Bay National Park"]["visitors"])
nikko_mean = np.mean(train[train["epark"]=="Nikko National Park"]["visitors"])
oyama_mean = np.mean(train[train["epark"]=="Oyama Oki National Park"]["visitors"])
towada_mean = np.mean(train[train["epark"]=="Towada Hachimantai National Park"]["visitors"])
mean_visitors = {"Akan Megumi National Park":akan_mean, "Aso Kuju National Park":aso_mean,
                 "Ise Shima National Park":ise_mean, "Kerama islands national park":kerama_mean,
                 "Kirishima Jinjiang Bay National Park":kirishima_mean, "Nikko National Park":nikko_mean,
                "Oyama Oki National Park":oyama_mean,"Towada Hachimantai National Park":towada_mean}

In [196]:
test_pred.head()

,index,datetime,epark,visitors
0,0,2015-01-01,Akan Megumi National Park,13469.371094
1,1,2015-01-02,Towada Hachimantai National Park,28518.166016
2,2,2015-01-03,Nikko National Park,23366.099609
3,3,2015-01-04,Ise Shima National Park,35663.039062
4,4,2015-01-05,Oyama Oki National Park,-759.467224


In [205]:
test_pred = test
test_pred["visitors"] = y_pred
for i in range(0, len(test_pred)):
    if test_pred.iloc[i]["visitors"]<0:
        test_pred.loc[i, "visitors"] = mean_visitors[test_pred.iloc[i]["epark"]]

In [206]:

submit = pd.DataFrame()

submit["prediction"] = test_pred["visitors"]
submit["prediction"] = submit["prediction"].astype(int)


In [207]:

submit.to_csv('/Users/abhishekkumar/Documents/MS_UB/Fall_18/ESL506_StatisticalDataMiningI/Project/SIGNATE/sample_submit.tsv',sep='\t',header=False)

In [137]:
set(train[train["visitors"]==0]["epark"])

{'Aso Kuju National Park',
 'Kerama islands national park',
 'Oyama Oki National Park'}

In [136]:
check = pd.DataFrame()
check = test
check["visitors"] = y_pred
set(check[check["visitors"]<0]["epark"])

{'Aso Kuju National Park',
 'Kerama islands national park',
 'Oyama Oki National Park'}

In [62]:
submit.index

RangeIndex(start=1, stop=2921, step=1)